# Compfest Bizzt 2025 - Bulu Kuduk Merinding


## Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px

## Model Penentu Barang

In [18]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from datetime import datetime

In [7]:
# Data
df_produk = pd.read_csv('data/produk_v3.csv')
df_toko = pd.read_csv('data/toko.csv')
df_transaksi = pd.read_csv('data/transaksi_v3.csv')


In [19]:
# 2.1. Konversi Tipe Data & Tentukan Tanggal Referensi
# Untuk konsistensi, kita anggap 'hari ini' adalah satu hari setelah transaksi terakhir di dataset
df_produk['expire_date'] = pd.to_datetime(df_produk['expire_date'])
df_transaksi['tanggal_transaksi'] = pd.to_datetime(df_transaksi['tanggal_transaksi'])
HARI_INI = df_transaksi['tanggal_transaksi'].max() + pd.Timedelta(days=1)
print(f"Tanggal referensi (HARI_INI) ditetapkan pada: {HARI_INI.date()}")

# 2.2. Membuat Fitur Agregat dari Tabel Transaksi
# Kita hitung metrik penjualan untuk setiap produk
print("\nMenghitung fitur agregat dari transaksi...")
agg_transaksi = df_transaksi.groupby('id_produk').agg(
    total_penjualan=('id_produk', 'count'),
    penjualan_terakhir=('tanggal_transaksi', 'max'),
    jumlah_hari_jual=('tanggal_transaksi', 'nunique')
).reset_index()

# Hitung penjualan harian rata-rata
# Tambah 1 untuk menghindari pembagian dengan nol
agg_transaksi['penjualan_harian_avg'] = agg_transaksi['total_penjualan'] / (agg_transaksi['jumlah_hari_jual'] + 1)
agg_transaksi['hari_sejak_penjualan_terakhir'] = (HARI_INI - agg_transaksi['penjualan_terakhir']).dt.days

print("Fitur agregat berhasil dibuat.")
display(agg_transaksi.head())


# 2.3. Membuat Fitur dari Tabel Produk & Menggabungkan Data
print("\nMenggabungkan semua data menjadi satu DataFrame...")
# Mulai dengan df_produk sebagai basis
df_model = df_produk.copy()

# Gabungkan dengan fitur agregat transaksi
df_model = pd.merge(df_model, agg_transaksi[['id_produk', 'penjualan_harian_avg', 'hari_sejak_penjualan_terakhir']], on='id_produk', how='left')

# Isi nilai NaN untuk produk yang mungkin belum pernah terjual
df_model['penjualan_harian_avg'].fillna(0, inplace=True)
# Jika belum pernah terjual, kita anggap sudah sangat lama
df_model['hari_sejak_penjualan_terakhir'].fillna(999, inplace=True)


# Buat fitur utama yang akan kita gunakan
df_model['hari_menuju_kedaluwarsa'] = (df_model['expire_date'] - HARI_INI).dt.days
# Fitur 'margin_headroom' mengukur seberapa banyak ruang yang kita miliki untuk diskon
# Kita asumsikan minimal_margin ada di data produk atau bisa di-join dari tabel kategori
# Untuk contoh ini, kita buat nilai dummy
df_model['minimal_margin'] = df_model['margin'] * 0.4 # Contoh: minimal margin adalah 40% dari margin saat ini
df_model['margin_headroom'] = df_model['margin'] - df_model['minimal_margin']

# Pilih fitur-fitur numerik yang relevan untuk model
fitur_model = [
    'margin',
    'hari_jual_minimal',
    'penjualan_harian_avg',
    'hari_sejak_penjualan_terakhir',
    'hari_menuju_kedaluwarsa',
    'margin_headroom'
]

# Tampilkan hasil akhir dari data yang siap dimodelkan
print("\nData akhir yang siap untuk pemodelan:")
display(df_model[['id_produk', 'nama_produk'] + fitur_model].head())

Tanggal referensi (HARI_INI) ditetapkan pada: 2025-03-01

Menghitung fitur agregat dari transaksi...
Fitur agregat berhasil dibuat.


,id_produk,total_penjualan,penjualan_terakhir,jumlah_hari_jual,penjualan_harian_avg,hari_sejak_penjualan_terakhir
0,P0000001,18,2025-02-22,17,1.000000,7
1,P0000002,15,2024-10-06,15,0.937500,146
2,P0000003,11,2025-01-19,11,0.916667,41
3,P0000004,19,2025-01-21,18,1.000000,39
4,P0000005,13,2024-12-30,13,0.928571,61



Menggabungkan semua data menjadi satu DataFrame...

Data akhir yang siap untuk pemodelan:

Data akhir yang siap untuk pemodelan:


C:\Users\USER\AppData\Local\Temp\ipykernel_21308\3387686918.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model['penjualan_harian_avg'].fillna(0, inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_21308\3387686918.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


,id_produk,nama_produk,margin,hari_jual_minimal,penjualan_harian_avg,hari_sejak_penjualan_terakhir,hari_menuju_kedaluwarsa,margin_headroom
0,P0000001,My Roti Roti Srikaya 350g,0.2005,2,1.000000,7.0,173,0.12030
1,P0000002,Pepsi Soda Original 500ml,0.1585,7,0.937500,146.0,530,0.09510
2,P0000003,Tango Biskuit Cokelat 50g,0.2241,14,0.916667,41.0,356,0.13446
3,P0000004,Frisian Flag Susu Moka 125ml,0.2286,5,1.000000,39.0,409,0.13716
4,P0000005,Delfi Cokelat Dark Chocolate 65g,0.1678,20,0.928571,61.0,529,0.10068


In [20]:
# ==============================================================================
# LANGKAH 3 (REVISI BESAR): Membuat Label Skor Urgensi Kontinu
# ==============================================================================
print("Membuat label 'urgency_score' yang lebih bernuansa...")

# Tentukan bobot bisnis (bisa disesuaikan)
W_KEDALUWARSA = 0.6  # Paling penting
W_KELAMBATAN = 0.3  # Cukup penting
W_PENJUALAN = 0.1   # Sebagai penalti

# Hindari pembagian dengan nol atau nilai negatif untuk hari menuju kedaluwarsa
# Kita batasi nilai minimumnya menjadi 1
df_model['hari_menuju_kedaluwarsa_safe'] = df_model['hari_menuju_kedaluwarsa'].clip(lower=1)

# Hitung komponen skor
# Skor kedaluwarsa (semakin kecil harinya, semakin besar skornya, non-linear)
df_model['skor_kedaluwarsa'] = 1 / df_model['hari_menuju_kedaluwarsa_safe']

# Skor kelambatan (semakin lama tidak laku, semakin tinggi skornya)
df_model['skor_kelambatan'] = df_model['hari_sejak_penjualan_terakhir']

# Skor penalti penjualan (semakin laku, semakin tinggi penaltinya)
df_model['skor_penalti_penjualan'] = df_model['penjualan_harian_avg']

# Gabungkan menjadi skor urgensi mentah
df_model['urgency_score_raw'] = (W_KEDALUWARSA * df_model['skor_kedaluwarsa'] +
                                 W_KELAMBATAN * df_model['skor_kelambatan'] -
                                 W_PENJUALAN * df_model['skor_penalti_penjualan'])

# Normalisasi skor mentah ke rentang 0-100 agar lebih mudah diinterpretasikan
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 100))
df_model['urgency_score'] = scaler.fit_transform(df_model[['urgency_score_raw']])

print("Distribusi 'urgency_score' yang baru:")
print(df_model['urgency_score'].describe())


# ==============================================================================
# LANGKAH 4 & 5 (REVISI BESAR): Melatih Model Regresi untuk Memprediksi Skor
# ==============================================================================
# Karena target kita sekarang kontinu, kita gunakan LGBMRegressor.
# Tujuannya adalah melatih model yang bisa memprediksi 'urgency_score' untuk produk baru
# atau saat data diperbarui.

# Definisikan fitur (X) dan target (y) BARU kita
X = df_model[fitur_model]  # Fitur tetap sama
y = df_model['urgency_score']   # Target sekarang adalah skor kontinu

# Bagi data menjadi set training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi {len(X_train)} baris training dan {len(X_test)} baris testing.")

# Inisialisasi model REGRESI
regressor = lgb.LGBMRegressor(
    objective='regression_l1',  # L1 (MAE) loss, lebih tahan terhadap outlier
    metric='mae',
    n_estimators=1000,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,
    num_leaves=31
)

print("\nMemulai pelatihan model LGBMRegressor...")
regressor.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='mae',
    callbacks=[lgb.early_stopping(20, verbose=False)]
)
print("Pelatihan model regresi selesai.")

# ==============================================================================
# LANGKAH 6 (REVISI BESAR): Membuat Peringkat Berdasarkan Prediksi Skor
# ==============================================================================
print("\nMemprediksi skor urgensi untuk semua produk...")

# Gunakan model regresi untuk memprediksi skor pada keseluruhan dataset
df_model['skor_prediksi'] = regressor.predict(X)

# Buat DataFrame hasil
df_hasil_peringkat_baru = df_model[[
    'id_produk', 'nama_produk', 'kategori_produk', 'skor_prediksi',
    'hari_menuju_kedaluwarsa', 'hari_sejak_penjualan_terakhir'
]].copy()

# Urutkan berdasarkan SKOR PREDIKSI dari yang tertinggi ke terendah
df_hasil_peringkat_baru = df_hasil_peringkat_baru.sort_values(by='skor_prediksi', ascending=False)

print("\n\n--- HASIL AKHIR (VERSI BARU): TOP 20 PRODUK KANDIDAT UNTUK DISKON ---")
display(df_hasil_peringkat_baru.head(20))

print("\nContoh sebaran skor prediksi yang unik:")
print(np.unique(df_hasil_peringkat_baru['skor_prediksi'].round(2)))

Membuat label 'urgency_score' yang lebih bernuansa...
Distribusi 'urgency_score' yang baru:
count    89898.000000
mean        29.578523
std         38.832976
min          0.000000
25%          3.411785
50%          8.923754
75%         33.659349
max        100.000000
Name: urgency_score, dtype: float64

Data dibagi menjadi 71918 baris training dan 17980 baris testing.

Memulai pelatihan model LGBMRegressor...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,000765 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1072
[LightGBM] [Info] Number of data points in the train set: 71918, number of used features: 6
[LightGBM] [Info] Start training from score 8,920397
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

,id_produk,nama_produk,kategori_produk,skor_prediksi,hari_menuju_kedaluwarsa,hari_sejak_penjualan_terakhir
89855,P0089856,Sari Roti Roti Tawar 200g,Roti,99.999992,171,999.0
89893,P0089894,Sari Roti Roti Srikaya 350g,Roti,99.999992,171,999.0
80129,P0080130,My Roti Roti Srikaya 70g,Roti,99.999992,171,999.0
84374,P0084375,Sari Roti Roti Pandan 350g,Roti,99.999992,171,999.0
74151,P0074152,My Roti Roti Keju 200g,Roti,99.999992,172,999.0
84334,P0084335,Mr. Bread Roti Pandan 200g,Roti,99.999992,171,999.0
89797,P0089798,Sari Roti Roti Keju 200g,Roti,99.999992,171,999.0
89828,P0089829,My Roti Roti Pandan 350g,Roti,99.999992,172,999.0
56662,P0056663,Mr. Bread Roti Pandan 350g,Roti,99.999992,171,999.0
74209,P0074210,Aoka Roti Keju 70g,Roti,99.999992,172,999.0



Contoh sebaran skor prediksi yang unik:
[0.000e+00 1.000e-02 2.000e-02 ... 6.263e+01 6.264e+01 1.000e+02]


In [22]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np # Import numpy

# Asumsikan df_hasil_peringkat_baru sudah berisi skor prediksi

# --- LANGKAH 1: Tentukan Parameter ---
TOTAL_SLOT_PROMOSI = 1000
SKOR_THRESHOLD = 50 # Angka ini bisa disesuaikan

# --- LANGKAH 2: Hitung Kandidat per Kategori ---
# Tandai produk mana yang dianggap 'kandidat kuat'
df_hasil_peringkat_baru['is_candidate'] = df_hasil_peringkat_baru['skor_prediksi'] > SKOR_THRESHOLD

# Hitung jumlah kandidat per kategori
candidate_counts = df_hasil_peringkat_baru[df_hasil_peringkat_baru['is_candidate']].groupby('kategori_produk').size()
total_candidates = candidate_counts.sum()

print("Jumlah kandidat kuat (skor > 50) per kategori:")
print(candidate_counts)

# --- LANGKAH 3: Hitung dan Alokasikan Kuota Dinamis ---
# Hitung kuota mentah berdasarkan proporsi
raw_quotas = (candidate_counts / total_candidates) * TOTAL_SLOT_PROMOSI

# Bulatkan kuota dan pastikan jumlahnya tepat 30 (logika pembulatan sederhana)
# Kita bulatkan ke bawah, lalu sisa slotnya kita berikan satu per satu ke kategori dengan desimal terbesar
quotas = np.floor(raw_quotas).astype(int) # Menggunakan np.floor()
sisa_slot = TOTAL_SLOT_PROMOSI - quotas.sum()

# Berikan sisa slot ke kategori dengan sisa desimal terbesar
sisa_desimal = raw_quotas - quotas
# Sort sisa desimal dari terbesar ke terkecil dan ambil indexnya
top_sisa_cats = sisa_desimal.sort_values(ascending=False).index.tolist()

# Alokasikan sisa slot
for cat in top_sisa_cats:
    if sisa_slot <= 0:
        break
    quotas[cat] += 1
    sisa_slot -= 1

# Pastikan setiap kategori yang punya kandidat dapat minimal 1 slot (opsional, tergantung aturan bisnis)
# Jika ada kategori dengan kandidat tapi kuota 0, alokasikan 1 (ambil dari kuota terbesar jika perlu)
# Untuk saat ini, kita biarkan bisa 0 jika proporsinya sangat kecil

print("\nKuota Dinamis yang Dialokasikan:")
print(quotas)

# --- LANGKAH 4: Ambil Produk Berdasarkan Kuota Dinamis ---
list_of_dfs = []
for category, num_to_take in quotas.items():
    if num_to_take > 0: # Hanya ambil jika kuotanya lebih dari 0
        # Ambil 'num_to_take' produk teratas dari kategori tersebut
        top_products_per_cat = (
            df_hasil_peringkat_baru[df_hasil_peringkat_baru['kategori_produk'] == category]
            .sort_values(by='skor_prediksi', ascending=False) # Pastikan diurutkan berdasarkan skor prediksi
            .head(num_to_take)
        )
        list_of_dfs.append(top_products_per_cat)

# Gabungkan semua hasilnya menjadi satu DataFrame
if list_of_dfs: # Cek apakah list tidak kosong sebelum concat
    df_final_dinamis = pd.concat(list_of_dfs)
else:
    df_final_dinamis = pd.DataFrame() # Buat DataFrame kosong jika tidak ada produk yang dipilih


# Urutkan hasil akhir berdasarkan skor
df_final_dinamis = df_final_dinamis.sort_values(by='skor_prediksi', ascending=False)

print(f"\n--- DAFTAR FINAL {len(df_final_dinamis)} PRODUK DENGAN KUOTA DINAMIS ---")
display(df_final_dinamis)

Jumlah kandidat kuat (skor > 50) per kategori:
kategori_produk
Biskuit    3154
Cokelat    3312
Roti       3531
Sirup      3396
Soda       3428
Susu       3530
dtype: int64

Kuota Dinamis yang Dialokasikan:
kategori_produk
Biskuit    155
Cokelat    163
Roti       174
Sirup      167
Soda       168
Susu       173
dtype: int64

--- DAFTAR FINAL 1000 PRODUK DENGAN KUOTA DINAMIS ---


,id_produk,nama_produk,kategori_produk,skor_prediksi,hari_menuju_kedaluwarsa,hari_sejak_penjualan_terakhir,is_candidate
73223,P0073224,My Roti Roti Keju 350g,Roti,99.999992,171,999.0,True
84985,P0084986,My Roti Roti Srikaya 200g,Roti,99.999992,171,999.0,True
73267,P0073268,Sari Roti Roti Srikaya 200g,Roti,99.999992,172,999.0,True
73265,P0073266,Mr. Bread Roti Srikaya 70g,Roti,99.999992,171,999.0,True
79947,P0079948,Aoka Roti Pandan 70g,Roti,99.999992,172,999.0,True
...,...,...,...,...,...,...,...
70407,P0070408,ABC Sirup Cocopandan 500ml,Sirup,99.999191,557,999.0,True
87753,P0087754,Tjap Buah Sirup Leci 500ml,Sirup,99.999191,555,999.0,True
73175,P0073176,Tjap Buah Sirup Jeruk 460ml,Sirup,99.999191,549,999.0,True
75415,P0075416,Tjap Buah Sirup Melon 500ml,Sirup,99.999191,552,999.0,True


## Model Penentu Diskon

In [23]:
# Muat semua data yang dibutuhkan
df_produk = pd.read_csv('data/produk_v3.csv')
df_toko = pd.read_csv('data/toko.csv')
df_transaksi = pd.read_csv('data/transaksi_v3.csv')
df_top_30_rekomendasi = df_final_dinamis.copy()

In [24]:
import pandas as pd
import numpy as np

print("--- Langkah 1: Membuat df_master_train (Menggabungkan data historis) ---")

# Menyeragamkan tipe data kunci untuk memastikan merge berhasil
df_transaksi['id_produk'] = df_transaksi['id_produk'].astype(str)
df_produk['id_produk'] = df_produk['id_produk'].astype(str)
df_toko['id_toko'] = df_toko['id_toko'].astype(int)
df_transaksi['id_toko'] = df_transaksi['id_toko'].astype(int)

# Gabungkan transaksi dengan data produk (menghapus kolom duplikat `id_toko` dari produk)
print("   - Menggabungkan transaksi dengan produk...")
if 'id_toko' in df_produk.columns:
    df_produk_cleaned = df_produk.drop(columns=['id_toko'])
else:
    df_produk_cleaned = df_produk
df_master_train = pd.merge(df_transaksi, df_produk_cleaned, on='id_produk', how='left')

# Gabungkan dengan data toko
print("   - Menggabungkan dengan data toko...")
df_master_train = pd.merge(df_master_train, df_toko, on='id_toko', how='left')

print("\n'df_master_train' (buku pelajaran untuk model) berhasil dibuat!")
print("Shape:", df_master_train.shape)
display(df_master_train.head())

--- Langkah 1: Membuat df_master_train (Menggabungkan data historis) ---
   - Menggabungkan transaksi dengan produk...
   - Menggabungkan dengan data toko...
   - Menggabungkan dengan data toko...

'df_master_train' (buku pelajaran untuk model) berhasil dibuat!
Shape: (605518, 38)

'df_master_train' (buku pelajaran untuk model) berhasil dibuat!
Shape: (605518, 38)


,id_produk,minggu,tanggal_transaksi,current_event,id_toko,harga_promosi,diskon,tipe_diskon,margin_promosi,hari_jual,...,jumlah_karyawan,tipe,umur_konsumen,jenis_kelamin_konsumen,penghasilan_konsumen_avg,pekerjaan_konsumen,kebiasaan_konsumen,reaksi_promo,frekuensi_pembelian,waktu_pembelian
0,P0006681,1,2023-01-01,Promo Akhir Pekan,1,5200,0.1215,Generic Product Discount,0.0769,164,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
1,P0005521,1,2023-01-01,Promo Akhir Pekan,1,13900,0.1852,Generic Product Discount,0.0072,103,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
2,P0001920,1,2023-01-01,Promo Akhir Pekan,1,49900,0.1563,Generic Product Discount,0.0762,114,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
3,P0002373,1,2023-01-01,Promo Akhir Pekan,1,11500,0.0675,Generic Product Discount,0.0696,20,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"
4,P0009121,1,2023-01-01,Promo Akhir Pekan,1,9700,0.0883,Generic Product Discount,0.1134,108,...,4,permukiman,"gen_alpha: 0.10, gen_z: 0.30, gen_y: 0.35, gen...","perempuan: 0.60, laki-laki: 0.40","terbatas: 0.25, menengah: 0.60, tinggi: 0.15","pelajar_mahasiswa: 0.25, pekerja_kantoran: 0.4...","impulsif: 0.40, terencana: 0.60","pemburu_promo: 0.35, oportunis: 0.45, loyal: 0.20","sering: 0.40, sesekali: 0.50, jarang: 0.10","pagi: 0.25, siang: 0.25, sore: 0.35, malam: 0.15"


In [31]:
print("\n--- Langkah 2: Mendefinisikan 'outcome' (profit_transaksi) ---")

df_master_train['profit_transaksi'] = df_master_train['harga_promosi'] - df_master_train['harga_beli']

print("Kolom 'profit_transaksi' berhasil dibuat di df_master_train.")


--- Langkah 2: Mendefinisikan 'outcome' (profit_transaksi) ---
Kolom 'profit_transaksi' berhasil dibuat di df_master_train.


In [32]:
print("\n--- Langkah 3: Feature Engineering pada Data Latihan ---")

# 3a. Parsing Fitur Konteks Toko
print("   - Mengekstrak fitur dari kolom Toko...")
try:
    df_master_train['rasio_pekerja_kantoran'] = df_master_train['pekerjaan_konsumen'].str.extract(r'pekerja_kantoran: (\d+\.\d+)').astype(float).fillna(0)
    df_master_train['rasio_impulsif'] = df_master_train['kebiasaan_konsumen'].str.extract(r'impulsif: (\d+\.\d+)').astype(float).fillna(0)
except Exception as e:
    print(f"   - Peringatan: Gagal mengekstrak fitur toko. Error: {e}")

# 3b. Membuat Fitur Baru dari Detail Produk
print("   - Membuat fitur 'gramasi' dan 'harga_per_gram'...")
df_master_train['gramasi'] = df_master_train['nama_produk'].str.extract(r'(\d+)\s?(g|ml)')[0].astype(float).fillna(0)
df_master_train['harga_per_gram'] = (df_master_train['harga_jual'] / df_master_train['gramasi']).replace([np.inf, -np.inf], 0).fillna(0)

# 3c. One-Hot Encoding Fitur Kategorikal
print("   - Melakukan One-Hot Encoding...")
df_master_train_featured = pd.get_dummies(df_master_train, columns=['brand', 'kategori_produk', 'current_event'], prefix=['brand', 'kat', 'event'])

print("Feature engineering pada data latihan selesai.")


--- Langkah 3: Feature Engineering pada Data Latihan ---
   - Mengekstrak fitur dari kolom Toko...
   - Membuat fitur 'gramasi' dan 'harga_per_gram'...
   - Membuat fitur 'gramasi' dan 'harga_per_gram'...
   - Melakukan One-Hot Encoding...
   - Melakukan One-Hot Encoding...
Feature engineering pada data latihan selesai.
Feature engineering pada data latihan selesai.


In [33]:
print("\n--- Langkah 4: Finalisasi dan Pemisahan Data Latihan ---")

# Definisikan semua nama kolom yang akan menjadi fitur (X)
fitur_numerik = [
    'harga_jual', 'margin', 'hari_jual', 'kedaluwarsa',
    'rasio_pekerja_kantoran', 'rasio_impulsif', 'gramasi', 'harga_per_gram'
]
fitur_ohe = [col for col in df_master_train_featured.columns if col.startswith(('brand_', 'kat_', 'event_'))]
feature_columns = fitur_numerik + fitur_ohe
print(f"   - Total {len(feature_columns)} fitur akan digunakan.")

# Pisahkan data menjadi beberapa bagian berdasarkan 'tipe_diskon'
dataframes_per_treatment = {}
for treatment_name in df_master_train_featured['tipe_diskon'].unique():
    dataframes_per_treatment[treatment_name] = df_master_train_featured[df_master_train_featured['tipe_diskon'] == treatment_name].copy()

print("\nData latihan telah dipisahkan per tipe diskon:")
for name, df in dataframes_per_treatment.items():
    print(f"- {name}: {len(df)} baris")


--- Langkah 4: Finalisasi dan Pemisahan Data Latihan ---
   - Total 47 fitur akan digunakan.

Data latihan telah dipisahkan per tipe diskon:
- Generic Product Discount: 180771 baris
- BOGO: 5708 baris
- Expired Discount: 91882 baris
- Tanpa Diskon: 231211 baris
- Discount Perkenalan: 4457 baris
- Event Based Discount: 91489 baris

Data latihan telah dipisahkan per tipe diskon:
- Generic Product Discount: 180771 baris
- BOGO: 5708 baris
- Expired Discount: 91882 baris
- Tanpa Diskon: 231211 baris
- Discount Perkenalan: 4457 baris
- Event Based Discount: 91489 baris


In [34]:
import lightgbm as lgb
print("\n--- Langkah 5: Melatih Model-model T-Learner ---")
trained_models = {}
for treatment_name, df_treatment in dataframes_per_treatment.items():
    print(f"--> Melatih model untuk: '{treatment_name}'")

    if len(df_treatment) < 100:
        print(f"    - Peringatan: Data terlalu sedikit, model untuk '{treatment_name}' dilewati.")
        continue

    X_train = df_treatment[feature_columns]
    y_train = df_treatment['profit_transaksi']

    model = lgb.LGBMRegressor(random_state=42, n_estimators=100, verbose=-1)
    model.fit(X_train, y_train)

    trained_models[treatment_name] = model
    print(f"    - Model untuk '{treatment_name}' berhasil dilatih.")

print("\n=================================================")
print("  PROSES PELATIHAN MODEL SPESIALIS SELESAI  ")
print("=================================================")


--- Langkah 5: Melatih Model-model T-Learner ---
--> Melatih model untuk: 'Generic Product Discount'
    - Model untuk 'Generic Product Discount' berhasil dilatih.
--> Melatih model untuk: 'BOGO'
    - Model untuk 'BOGO' berhasil dilatih.
--> Melatih model untuk: 'Expired Discount'
    - Model untuk 'Generic Product Discount' berhasil dilatih.
--> Melatih model untuk: 'BOGO'
    - Model untuk 'BOGO' berhasil dilatih.
--> Melatih model untuk: 'Expired Discount'
    - Model untuk 'Expired Discount' berhasil dilatih.
--> Melatih model untuk: 'Tanpa Diskon'
    - Model untuk 'Expired Discount' berhasil dilatih.
--> Melatih model untuk: 'Tanpa Diskon'
    - Model untuk 'Tanpa Diskon' berhasil dilatih.
--> Melatih model untuk: 'Discount Perkenalan'
    - Model untuk 'Discount Perkenalan' berhasil dilatih.
--> Melatih model untuk: 'Event Based Discount'
    - Model untuk 'Tanpa Diskon' berhasil dilatih.
--> Melatih model untuk: 'Discount Perkenalan'
    - Model untuk 'Discount Perkenalan' be

In [35]:
print("--- Langkah 6: Menyiapkan Data untuk Diberi Rekomendasi (Versi Perbaikan) ---")

# Ambil daftar ID produk dari hasil Model 1
list_id_rekomendasi = df_top_30_rekomendasi['id_produk'].astype(str).unique().tolist()

# Mulai dengan DataFrame yang berisi produk-produk ini beserta detailnya dari df_produk
df_rekomendasi = df_produk[df_produk['id_produk'].isin(list_id_rekomendasi)].copy()

# Gabungkan dengan data toko
df_rekomendasi = pd.merge(df_rekomendasi, df_toko, on='id_toko', how='left')

print(f"   - Menyiapkan data untuk {len(df_rekomendasi)} kombinasi produk-toko...")

# ============================ SOLUSI DI SINI ============================
# [SOLUSI] Tambahkan kolom 'current_event' secara manual untuk simulasi.
# Kita asumsikan promosi akan dijalankan pada konteks 'Promo Akhir Pekan'.
df_rekomendasi['current_event'] = 'Promo Akhir Pekan'
print("   - Kolom 'current_event' ditambahkan dengan nilai 'Promo Akhir Pekan'.")
# =======================================================================


# Lakukan feature engineering yang SAMA PERSIS dengan Langkah 3
print("   - Menerapkan feature engineering...")

# 6a. Parsing Fitur Konteks Toko
try:
    df_rekomendasi['rasio_pekerja_kantoran'] = df_rekomendasi['pekerjaan_konsumen'].str.extract(r'pekerja_kantoran: (\d+\.\d+)').astype(float).fillna(0)
    df_rekomendasi['rasio_impulsif'] = df_rekomendasi['kebiasaan_konsumen'].str.extract(r'impulsif: (\d+\.\d+)').astype(float).fillna(0)
except Exception: pass

# 6b. Membuat Fitur Baru dari Detail Produk
df_rekomendasi['gramasi'] = df_rekomendasi['nama_produk'].str.extract(r'(\d+)\s?(g|ml)')[0].astype(float).fillna(0)
df_rekomendasi['harga_per_gram'] = (df_rekomendasi['harga_jual'] / df_rekomendasi['gramasi']).replace([np.inf, -np.inf], 0).fillna(0)
df_rekomendasi['hari_jual'] = df_rekomendasi['hari_jual_minimal']
df_rekomendasi['kedaluwarsa'] = 365 - df_rekomendasi['hari_jual_minimal']


# 6c. One-Hot Encoding (Sekarang akan berhasil)
df_rekomendasi_featured = pd.get_dummies(df_rekomendasi, columns=['brand', 'kategori_produk', 'current_event'], prefix=['brand', 'kat', 'event'])

# 6d. Finalisasi - Pastikan semua kolom ada dan urutannya sama
for col in feature_columns:
    if col not in df_rekomendasi_featured.columns:
        df_rekomendasi_featured[col] = 0
X_rekomendasi = df_rekomendasi_featured[feature_columns]

print("Data rekomendasi siap untuk diprediksi.")

--- Langkah 6: Menyiapkan Data untuk Diberi Rekomendasi (Versi Perbaikan) ---
   - Menyiapkan data untuk 1000 kombinasi produk-toko...
   - Kolom 'current_event' ditambahkan dengan nilai 'Promo Akhir Pekan'.
   - Menerapkan feature engineering...
Data rekomendasi siap untuk diprediksi.


In [45]:
print("\n--- Langkah 7: Simulasi, Perhitungan Uplift, dan Rekomendasi Final (Versi Perbaikan) ---")

# ==============================================================================
# Tahap 1 & 2 (Tidak berubah)
# ==============================================================================
print("   - Tahap 1 & 2: Menyiapkan data dan melakukan prediksi...")
# (Kode dari langkah 6 untuk menyiapkan X_rekomendasi diasumsikan sudah berjalan)

hasil_prediksi = {}
for treatment_name, model in trained_models.items():
    prediksi = model.predict(X_rekomendasi)
    hasil_prediksi[treatment_name] = prediksi

df_prediksi_profit = pd.DataFrame(hasil_prediksi, index=df_rekomendasi.index)


# ==============================================================================
# Tahap 3: Hitung Uplift dan Tentukan Strategi Terbaik (Logika Diperbaiki)
# ==============================================================================
print("   - Tahap 3: Menghitung uplift dan memilih strategi terbaik...")

if 'Tanpa Diskon' in df_prediksi_profit.columns:
    baseline_profit = df_prediksi_profit['Tanpa Diskon']
    # df_uplift_numeric hanya berisi kolom-kolom uplift yang berupa angka
    df_uplift_numeric = df_prediksi_profit.drop(columns=['Tanpa Diskon'], errors='ignore').subtract(baseline_profit, axis=0)
else:
    print("   - Peringatan: Model 'Tanpa Diskon' tidak ditemukan.")
    df_uplift_numeric = df_prediksi_profit
    baseline_profit = 0

# [DIUBAH] Buat DataFrame hasil untuk menampung rekomendasi
df_hasil_rekomendasi = pd.DataFrame(index=df_uplift_numeric.index)

# [DIUBAH] 1. Cari nama strategi terbaik DULU (dari DataFrame angka)
df_hasil_rekomendasi['rekomendasi_strategi'] = df_uplift_numeric.idxmax(axis=1)

# [DIUBAH] 2. BARU ambil nilai uplift tertinggi
df_hasil_rekomendasi['estimasi_uplift_profit'] = df_uplift_numeric.max(axis=1)

# 3. Logika untuk menangani uplift negatif (tidak berubah)
df_hasil_rekomendasi.loc[df_hasil_rekomendasi['estimasi_uplift_profit'] < 0, 'rekomendasi_strategi'] = 'Tanpa Diskon'
df_hasil_rekomendasi.loc[df_hasil_rekomendasi['estimasi_uplift_profit'] < 0, 'estimasi_uplift_profit'] = 0


# ==============================================================================
# Tahap 4: Tambahkan Informasi Besaran Diskon dari Data Historis
# ==============================================================================
print("   - Tahap 4: Menambahkan informasi besaran diskon dari data historis...")

# Hitung rata-rata diskon untuk setiap strategi dari data historis
diskon_rata_rata = df_master_train.groupby('tipe_diskon')['diskon'].agg(['mean', 'std']).reset_index()
diskon_rata_rata.columns = ['tipe_diskon', 'avg_diskon', 'std_diskon']

print("Rata-rata besaran diskon per strategi dari data historis:")
display(diskon_rata_rata)

# Buat mapping besaran diskon
diskon_mapping = dict(zip(diskon_rata_rata['tipe_diskon'], diskon_rata_rata['avg_diskon']))

# Function untuk format diskon
def format_discount_info(strategi, diskon_pct):
    if strategi == 'Tanpa Diskon':
        return '0%', 'Tidak ada diskon'
    elif strategi == 'BOGO':
        return 'BOGO', 'Buy 1 Get 1 Free'
    else:
        pct = diskon_pct * 100
        return f'{pct:.1f}%', f'Diskon {pct:.1f}%'

# Tambahkan informasi besaran diskon ke hasil rekomendasi
df_hasil_rekomendasi['besaran_diskon_pct'] = df_hasil_rekomendasi['rekomendasi_strategi'].map(diskon_mapping).fillna(0)
df_hasil_rekomendasi['besaran_diskon'] = df_hasil_rekomendasi.apply(
    lambda row: format_discount_info(row['rekomendasi_strategi'], row['besaran_diskon_pct'])[0], axis=1
)
df_hasil_rekomendasi['deskripsi_diskon'] = df_hasil_rekomendasi.apply(
    lambda row: format_discount_info(row['rekomendasi_strategi'], row['besaran_diskon_pct'])[1], axis=1
)


# ==============================================================================
# Tahap 5: Tampilkan Hasil Akhir dengan Besaran Diskon
# ==============================================================================
print("\n============================================================")
print("         HASIL AKHIR: REKOMENDASI STRATEGI DISKON LENGKAP")
print("============================================================")

df_final_recommendation = pd.concat([
    df_rekomendasi[['id_produk', 'nama_produk', 'kategori_produk', 'id_toko', 'harga_jual']].reset_index(drop=True),
    df_hasil_rekomendasi[['rekomendasi_strategi', 'besaran_diskon', 'deskripsi_diskon', 'estimasi_uplift_profit']].reset_index(drop=True)
], axis=1)

# Hitung harga setelah diskon
df_final_recommendation['harga_setelah_diskon'] = df_final_recommendation.apply(
    lambda row: row['harga_jual'] if row['besaran_diskon'] == '0%' or row['besaran_diskon'] == 'BOGO'
    else row['harga_jual'] * (1 - float(row['besaran_diskon'].replace('%', '')) / 100), axis=1
).round(0).astype(int)

# Format harga untuk display
df_final_recommendation['harga_display'] = df_final_recommendation.apply(
    lambda row: f"Rp {row['harga_jual']:,}" if row['besaran_diskon'] == '0%'
    else f"Rp {row['harga_jual']:,} → BOGO" if row['besaran_diskon'] == 'BOGO'
    else f"Rp {row['harga_jual']:,} → Rp {row['harga_setelah_diskon']:,}", axis=1
)

# Agregasi per produk
df_agg_recommendation_enhanced = df_final_recommendation.groupby(['id_produk', 'nama_produk', 'kategori_produk']).agg(
    rekomendasi_utama=('rekomendasi_strategi', lambda x: x.mode()[0] if not x.empty else "N/A"),
    besaran_diskon_utama=('besaran_diskon', lambda x: x.mode()[0] if not x.empty else "0%"),
    deskripsi_diskon_utama=('deskripsi_diskon', lambda x: x.mode()[0] if not x.empty else "Tidak ada"),
    harga_display_utama=('harga_display', lambda x: x.mode()[0] if not x.empty else "N/A"),
    rata_rata_uplift_profit=('estimasi_uplift_profit', 'mean')
).reset_index().sort_values(by='rata_rata_uplift_profit', ascending=False)

# Display hasil dengan format yang menarik
display(df_agg_recommendation_enhanced.head(30).style.format({
    'rata_rata_uplift_profit': 'Rp {:,.0f}'.format
}).background_gradient(subset=['rata_rata_uplift_profit'], cmap='Greens'))


--- Langkah 7: Simulasi, Perhitungan Uplift, dan Rekomendasi Final (Versi Perbaikan) ---
   - Tahap 1 & 2: Menyiapkan data dan melakukan prediksi...
   - Tahap 3: Menghitung uplift dan memilih strategi terbaik...
   - Tahap 4: Menambahkan informasi besaran diskon dari data historis...
Rata-rata besaran diskon per strategi dari data historis:


,tipe_diskon,avg_diskon,std_diskon
0,BOGO,0.500000,0.000000
1,Discount Perkenalan,0.174631,0.043193
2,Event Based Discount,0.116654,0.078345
3,Expired Discount,0.400036,0.057572
4,Generic Product Discount,0.124842,0.043313
5,Tanpa Diskon,0.000000,0.000000



         HASIL AKHIR: REKOMENDASI STRATEGI DISKON LENGKAP


,id_produk,nama_produk,kategori_produk,rekomendasi_utama,besaran_diskon_utama,deskripsi_diskon_utama,harga_display_utama,rata_rata_uplift_profit
195,P0071759,Indomilk Susu Cokelat 1000ml,Susu,Event Based Discount,11.7%,Diskon 11.7%,"Rp 16,400 → Rp 14,481",Rp 363
185,P0071584,Khong Guan Biskuit Cokelat 50g,Biskuit,BOGO,BOGO,Buy 1 Get 1 Free,"Rp 1,000 → BOGO",Rp 238
0,P0027924,Ultra Milk Susu Full Cream 1000ml,Susu,Event Based Discount,11.7%,Diskon 11.7%,"Rp 17,400 → Rp 15,364",Rp 232
479,P0077715,Ultra Milk Susu Moka 1000ml,Susu,Event Based Discount,11.7%,Diskon 11.7%,"Rp 18,200 → Rp 16,071",Rp 218
739,P0084264,Diamond Susu Vanila 1000ml,Susu,Event Based Discount,11.7%,Diskon 11.7%,"Rp 37,300 → Rp 32,936",Rp 187
742,P0084291,Oreo Biskuit Kelapa 50g,Biskuit,BOGO,BOGO,Buy 1 Get 1 Free,"Rp 1,000 → BOGO",Rp 160
691,P0083164,Diamond Susu Vanila 125ml,Susu,Event Based Discount,11.7%,Diskon 11.7%,"Rp 2,200 → Rp 1,943",Rp 158
640,P0081742,Tango Biskuit Cokelat 50g,Biskuit,BOGO,BOGO,Buy 1 Get 1 Free,"Rp 1,400 → BOGO",Rp 135
328,P0074251,Tango Biskuit Kelapa 50g,Biskuit,BOGO,BOGO,Buy 1 Get 1 Free,"Rp 1,400 → BOGO",Rp 135
431,P0076198,Tango Biskuit Kelapa 50g,Biskuit,BOGO,BOGO,Buy 1 Get 1 Free,"Rp 1,400 → BOGO",Rp 135
